In [1]:
import time
import os
import numpy as np
import pandas as pd
from collections import Counter
from nltk.stem import WordNetLemmatizer
start = time.time()

## ALL 20 folder names

In [2]:
groups_for_train = ['alt.atheism', 'comp.graphics' , 'comp.os.ms-windows.misc' , 'comp.sys.ibm.pc.hardware' , 'comp.sys.mac.hardware' , 'comp.windows.x' , 'misc.forsale' , 'rec.autos' , 'rec.motorcycles' , 'rec.sport.baseball' , 'rec.sport.hockey' , 'sci.crypt' , 'sci.electronics' , 'sci.med' , 'sci.space' , 'soc.religion.christian' , 'talk.politics.guns' , 'talk.politics.mideast' , 'talk.politics.misc' ,'talk.religion.misc' ]
len(groups_for_train)

20

## Function for Stop_words

In [3]:
def  Remove_stop_words(x):
    l=len(x)
    
    y=[]
    stop_words=['a','for','Not','not','NOT','are','when','were','then','them','some','also','just','1993','as','on','if','will','here','there','their','with','will','all','what','at','my','who','but','so','or','would','can','be','i','you','by','your','we','was','an','that','they','which','the','in','are,''that','he','she','should','you','yours','it','has','have','is','this','had','could','from','to','of','and','an','A','For','Are','When','Were','Then','Them','Some','Also','Just','1993','As','On','If','Will','Here','There','Their','With','Will','All','What','At','My','Who','But','So','Or','Would','Can','Be','I','You','By','Your','We','Was','An','That','They','Which','The','In','Are,''That','He','She','Should','You','Yours','It','Has','Have','Is','This','Had','Could','From','To','of','And','An']
    
    for i in range(l):
        if x[i] not in stop_words :
            y.append(x[i])
    
    return y

## Function for lower case all extracted words

In [4]:
def lower_casing(words): 
    text=[]
    for i in words :
        text.append(i.lower())
    return text

## Function for Lemmatization (using nltk)

In [5]:
def lemmatization(words):
    lemmatizer = WordNetLemmatizer()
    x=[]
    for i in words:
        x.append(lemmatizer.lemmatize(i))
    return x

## Function for Tokenization

In [6]:
def tokenizer(words):
    for i in range(len(words)):
            if not words[i].isalpha() :       # making all the words as empty string which are not alphabets
                words[i] = ''
    new_words=[]
    for i in words :   
        if (i != ''):
            new_words.append(i)               # now getting only alphabets from the words list 

    new_words = lower_casing(new_words)       # lower casing all the words , ex: (Good --> good)
    
    return new_words
    

## Function for getting words greater than length of 3

In [7]:
def size_changer(words):
    x=[]
    for i in words:
        if (len(i) > 3):       # taking only those words whose length is greater than 3
            x.append(i)
    return x
        

## Function for extracting words only for vocabulary(only from train data)

In [8]:
def getting_data_for_vocab(x):
    direct = r'20 news group dataset\Train data\\' + x + '/'
    files = os.listdir(direct)

    alt = [direct + i for i in files]           # extracting all files from a particular folders x( from trainging data)
    words=[]
    
    for j in alt :
        f = open(j)
        reading = f.read()                      # extracting all text from all files
        words +=reading.split(' ')
     
    
    new_words = tokenizer(words)                # applying tokenization 
    
    new_words = lemmatization(new_words)        # applying lemmatization
   
    return new_words

## Function for extracting top most common words from words list (using Counter Function)

In [9]:

def make_dictionary(x):
    words=x
    words_without_stop_words = Remove_stop_words(words)                        # removing stop words
    
    words_with_length_greater_than_3 = size_changer(words_without_stop_words)  # take out all words greater than 3
    
    dictionary = Counter(words_with_length_greater_than_3)                     # using (counter function)
    
    x = dictionary.most_common(5000)                                        
    
    return x
  
    

## Function call for vocabulary

In [10]:
vocab=[]
for i in groups_for_train:
    vocab += getting_data_for_vocab(i)

## Fucntion call for Dictionary

In [11]:
My_dictionary=make_dictionary(vocab)
type(My_dictionary)

list

In [12]:
len(My_dictionary)

5000

## Function  for making documnet for Train Data (with applying all functions)

In [13]:
def getting_docs_for_train(x,y):
    direct = r'20 news group dataset\Train data\\' + x + '/'
    
    files = os.listdir(direct)                           # extracting all files from a particular folder x(from train data)

    alt = [direct + i for i in files]         
    words=[]
    
    data = []                                       
    y_values = []
    for j in alt :                                      # extracting all text from all files and make a seperate row for every files 
        f = open(j)                                        
        reading = f.read()                                 
        words =reading.split(' ')                         
        
        new_words = tokenizer(words)                    # applying tokenizer on the text of particular file
        
        new_words = lemmatization(new_words)            # applying lemmatization on the text of particular file
        
        new_words = Remove_stop_words(new_words)        # removing stop words from text of particular file
        
        new_words = size_changer(new_words)             # taking all words with lenth greater than 3
        
        data.append(new_words)                          # appending the list of words of a particular file as a row in data(it becomes a 2d list)
        y_values.append(y)                              # appending the number of folder which the data belongs 

    return data,y_values

## Loop which calls function(getting_docs_for_train) for every folder(groups_for_train)

In [14]:
data = []
y=0
Y_train=[]
for i in groups_for_train :                       # loops for every folder
    datas=0
    y_val=0
    datas,y_val = getting_docs_for_train(i,y)     # function call on every folder (which is 20)
    data += datas
    Y_train += y_val                             
    y += 1
print(len(data))
print(len(Y_train))

16315
16315


## Loop for creating actual dataset for x_train

In [15]:
print(type(data))
print(type(My_dictionary))
X_train= np.zeros((len(data),len(My_dictionary)))    # creating a dummy datasets with all zeros

for row in range(len(data)):                         
    for word in data[row]:
        for column in range(len(My_dictionary)):
            if(word==My_dictionary[column][0]):      # if that words is matching with the current feature words
                X_train[row][column]+=1              # then we will increment that place by one

<class 'list'>
<class 'list'>


## Function  for making documnet for Test Data (with applying all functions)

### Note: all functions are same as i did for train data

In [16]:
def getting_docs_for_test(x,y):
    direct = r'20 news group dataset\Test data\\' + x + '/'
    files = os.listdir(direct)

    alt = [direct + i for i in files]
    words=[]

    data = []
    y_values = []
    for j in alt :
        f = open(j)
        reading = f.read()
        words =reading.split(' ')
        new_words = tokenizer(words)
        new_words = lemmatization(new_words)
        new_words = Remove_stop_words(new_words)
        new_words = size_changer(new_words)
        data.append(new_words)
        y_values.append(y)

    return data,y_values

In [17]:
groups_for_test=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19']

## Loop which calls function(getting_docs_for_test) for every folder(groups_for_test)

### Note: all is same ass i did for train data

In [18]:
data_for_test = []
y=0
Y_test=[]
for i in groups_for_test :
    datas=0
    y_val=0
    datas,y_val = getting_docs_for_test(i,y)
    data_for_test += datas
    Y_test += y_val
    y += 1
print(len(data_for_test))
print(len(Y_test))


3682
3682


## Loop for creating actual dataset for x_train

### Note: all is same as i did for train dataset

In [19]:
print(type(data_for_test))
print(type(My_dictionary))
X_test= np.zeros((len(data_for_test),len(My_dictionary)))

for row in range(len(data_for_test)):
    for word in data_for_test[row]:
        for column in range(len(My_dictionary)):
            if(word==My_dictionary[column][0]):
                X_test[row][column]+=1

<class 'list'>
<class 'list'>


In [20]:
print(X_train.shape)
print(X_test.shape)

(16315, 5000)
(3682, 5000)


## From here i am applying Naive_Bayes classsifier ( Inbuilt)

In [21]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


In [22]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=1)
clf.fit(X_train,Y_train)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

##  Implementing 
### 1.  Confusion matrix
### 2.  Classification report

In [23]:
y_test_pred=clf.predict(X_test)

print("Confusion Matrix is:")
print()
print(confusion_matrix(Y_test,y_test_pred))

print()

print("Classification report is:")
print()
print(classification_report(Y_test,y_test_pred))

clf.score(X_test,Y_test)

Confusion Matrix is:

[[129   1   0   0   0   0   0   0   1   0   1   2   0   1   0  11   1   0
    2  19]
 [  1 118  14  12   2  11   4   4   0   0   0   0   3   0   1   0   0   1
    0   0]
 [  1   7 133  19   8  16   4   1   0   0   1   1   1   0   0   0   0   0
    0   0]
 [  1   4  15 129  19   3   3   4   0   0   1   0   9   0   0   0   0   1
    1   0]
 [  0   1   9  25 122   0  10   1   1   0   1   0   6   0   0   0   0   0
    0   0]
 [  1  17  11   6   0 132   1   2   0   1   0   1   1   2   0   0   0   0
    0   0]
 [  0   1   2   7   9   0 134  11   2   0   2   1   5   0   1   0   1   1
    0   0]
 [  0   0   0   1   2   1   5 125  15   0   1   4   6   3   0   1   7   0
    2   1]
 [  1   0   0   0   3   0   6  19 142   0   0   0   2   1   1   0   4   0
    1   2]
 [  0   0   1   0   1   1   4   1   5 171   6   1   1   1   1   0   2   1
    1   0]
 [  0   1   1   0   1   0   1   0   3  10 177   0   1   3   0   1   0   0
    1   0]
 [  1   3   3   0   1   3   0   0   1   0  

0.7403585008147746

## From here i am applying Naive_bayes classifier( Own )

In [24]:
class Own_MultinomialNB(object):
    def __init__(self, alpha=1):
        self.alpha = alpha

    def fit(self, X, y):
        count_sample = X.shape[0]
        separated = [[x for x, t in zip(X, y) if t == c] for c in np.unique(y)]
        self.class_log_prior_ = [np.log(len(i) / count_sample) for i in separated]
        count = np.array([np.array(i).sum(axis=0) for i in separated]) + self.alpha
        self.feature_log_prob_ = np.log(count / count.sum(axis=1)[np.newaxis].T)
        return self

    def predict_log(self, X):
        return [(self.feature_log_prob_ * x).sum(axis=1) + self.class_log_prior_
                for x in X]

    def predict(self, X):
        return np.argmax(self.predict_log(X), axis=1)

In [25]:
clf_own = Own_MultinomialNB()
clf_own.fit(X_train, Y_train)

In [26]:
y_test_pred1=clf_own.predict(X_test)

print("Confusion Matrix is:")
print()
print(confusion_matrix(Y_test,y_test_pred1))

print()

print("Classification report is:")
print()
print(classification_report(Y_test,y_test_pred1))


Confusion Matrix is:

[[129   1   0   0   0   0   0   0   1   0   1   2   0   1   0  11   1   0
    2  19]
 [  1 118  14  12   2  11   4   4   0   0   0   0   3   0   1   0   0   1
    0   0]
 [  1   7 133  19   8  16   4   1   0   0   1   1   1   0   0   0   0   0
    0   0]
 [  1   4  15 129  19   3   3   4   0   0   1   0   9   0   0   0   0   1
    1   0]
 [  0   1   9  25 122   0  10   1   1   0   1   0   6   0   0   0   0   0
    0   0]
 [  1  17  11   6   0 132   1   2   0   1   0   1   1   2   0   0   0   0
    0   0]
 [  0   1   2   7   9   0 134  11   2   0   2   1   5   0   1   0   1   1
    0   0]
 [  0   0   0   1   2   1   5 125  15   0   1   4   6   3   0   1   7   0
    2   1]
 [  1   0   0   0   3   0   6  19 142   0   0   0   2   1   1   0   4   0
    1   2]
 [  0   0   1   0   1   1   4   1   5 171   6   1   1   1   1   0   2   1
    1   0]
 [  0   1   1   0   1   0   1   0   3  10 177   0   1   3   0   1   0   0
    1   0]
 [  1   3   3   0   1   3   0   0   1   0  

In [27]:
clf.score(X_test,Y_test)


0.7403585008147746

In [28]:
clf.score(X_train, Y_train)

0.8397793441618143

In [29]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(clf,X_test,Y_test))
print('time is :',(time.time()-start)/60, 'mins')

[0.65746753 0.69543974 0.6685761 ]
time is : 29.508705174922945 mins


## Note : increasing number of features it gives better score 